In [1]:
#uncomment all lines below in this cell to replicate my installation on colab

!pip install qeds fiona geopandas xgboost gensim folium pyLDAvis descartes psaw pyarrow
!apt-get install libproj-dev proj-data proj-bin  
!apt-get install libgeos-dev  
!pip install cython  
!pip install cartopy 
!brew install proj geos
!pip3 uninstall shapely
!pip3 install --upgrade cython numpy pyshp six
!pip3 install shapely --no-binary shapely
!pip3 install git+https://github.com/SciTools/cartopy.git --no-binary cartopy
!pip install fuzzywuzzy
!pip install python-Levenshtein


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libproj-dev is already the newest version (4.9.3-2).
proj-bin is already the newest version (4.9.3-2).
proj-data is already the newest version (4.9.3-2).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgeos-dev is already the newest version (3.6.2-1build2).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/bin/bash: brew: command not found
Uninstalling Shapely-1.7.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Shapely-1.7.0-py3.6.egg-info
    /usr/local/lib/python3.6/dist-packages/shapely/*
Proceed (y/n)? y
  Successfully uninstalled Shapely-1.7.0
Requirement already up-to-date: cython in /usr/local/lib/python3.6/dist-packages (0.29.19)
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.18.5)
Requirement already up-to

In [0]:
import pandas as pd 
import os
import IPython

import json
#with open("to_from_data.json", "r") as read_file:
   #rankings = json.load(read_file)
#data = pd.DataFrame(rankings)

In [0]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import qeds
qeds.themes.mpl_style();

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [0]:
ranking_data = pd.read_json("tilburg_2019_aa_dsouza.json") #set directory as necessary
pd.set_option('display.max_rows', ranking_data.shape[0]+1)

#input n x 2 dataframe with columns renamed to "rank", "institution" such that "rank" dtype is int64 and "institution" dtype is string

In [0]:
ranking_data["rank"] = ranking_data.index + 1 #give rank number to all institutions, ties resolved alphabetically
ranking_data = ranking_data.rename(columns = {"University": "institution"})
ranking_data["institution"] = ranking_data["institution"].astype(str)
ranking_data = ranking_data[["rank", "institution"]]

In [6]:
ranking_data

,rank,institution
0,1,STANFORD UNIVERSITY
1,2,LONDON SCHOOL OF ECONOMICS AND POLITICAL SCIENCE
2,3,UNIVERSITY OF CHICAGO
3,4,MASSACHUSETTS INSTITUTE OF TECHNOLOGY
4,5,HARVARD UNIVERSITY
5,6,PRINCETON UNIVERSITY
6,7,"UNIVERSITY OF CALIFORNIA, BERKELEY"
7,8,NORTHWESTERN UNIVERSITY
8,9,UNIVERSITY OF PENNSYLVANIA
9,10,UNIVERSITY OF OXFORD


In [0]:
ranking_data = ranking_data.applymap(lambda i:i.lower() if type(i) == str else i)  #lowercase strings

In [0]:
organizations_data = pd.read_json("organizations_at_ejm.json")

In [0]:
organizations_data = organizations_data.applymap(lambda i:i.lower() if type(i) == str else i)  #lowercase strings

In [0]:
organizations_name_list = organizations_data["name"].tolist() #list of organization names

In [0]:
rankings_foo_process = ranking_data["institution"].apply(lambda x : process.extract(x, organizations_name_list, limit = 1)) #series of organization names matched with ranking data institution; format [(organization_name, levenshtein score)]

In [0]:
ranking_data["name"] = rankings_foo_process #create column using series

In [0]:
ranking_data["name"] = pd.DataFrame(ranking_data.name.tolist(), index = ranking_data.index) #0th element of list

In [0]:
ranking_data[["name", "match_score"]] = pd.DataFrame(ranking_data.name.tolist(), index = ranking_data.index) #split tuple into elements in two columns

In [0]:
ranking_data_perfect_match = ranking_data[ranking_data["match_score"] == 100]

In [0]:
ranking_data_imperfect_match = ranking_data[ranking_data["match_score"] != 100]

In [17]:
ranking_data_imperfect_match

,rank,institution,name,match_score
12,13,bocconi university,bocconi university (università bocconi),90
20,21,"university of california, los angeles",university of california los angeles (ucla),95
24,25,university of toulouse / toulouse school of ec...,toulouse school of economics,90
28,29,"queen mary, university of london",queen mary university of london,98
29,30,university of amsterdam,university of amsterdam (universiteit van amst...,90
33,34,university of zurich,university of zurich (universität zürich),90
35,36,eth zurich,university of zurich (universität zürich),86
36,37,pompeu fabra university,universitat pompeu fabra,89
38,39,texas a&m university,west texas a&m university,95
41,42,"university of texas, austin",university of texas at austin,96


In [0]:
#add condition for threshold

In [0]:
ranking_with_ejmid = pd.merge(ranking_data, organizations_data, on = "name", how = "left")

In [20]:
ranking_with_ejmid

,rank,institution,name,match_score,department,shortname,oid,institution_id
0,1,stanford university,stanford university,100,department of economics,"economics, stanford university",92,78
1,1,stanford university,stanford university,100,stanford institute for economic policy researc...,"policy research, stanford univ",130,78
2,1,stanford university,stanford university,100,"economics, graduate school of business","economics (business), stanford u",598,78
3,1,stanford university,stanford university,100,rural education action program,"reap, stanford university",865,78
4,1,stanford university,stanford university,100,center on food security and the environment,"food security, stanford univ",1167,78
...,...,...,...,...,...,...,...,...
916,110,university of vienna,university of vienna (universität wien),90,faculty of mathematics,"faculty of math, univ vienna",3557,254
917,110,university of vienna,university of vienna (universität wien),90,research institute for family business,research institute for family bu,4248,254
918,110,university of vienna,university of vienna (universität wien),90,faculty of psychology,"fac psychology, univ vienna",5022,254
919,111,xiamen university,xiamen university,100,wang yanan institute for studies in economics,"wise&soe, xiamen university",514,445


In [0]:
#ranking_with_ejmid.to_json("ranking_with_ejmid.json") #export to json